In [4]:
CSV_IN = 'datos/Base datos_INTA_ascii_limpio.csv'

In [5]:
print 'Cargando datos de "%s"' % CSV_IN
df = pd.read_csv(CSV_IN)
print 'Cargado dataframe de tamaño %d x %d' % df.shape

Cargando datos de "datos/Base datos_INTA_ascii_limpio.csv"
Cargado dataframe de tamaño 423 x 69


In [6]:
def truncate_str(s, n=15):
    s = str(s)
    if len(s) > n:
        return s[:(n-3)] + '...'
    else:
        return s

def smart_value_counts(s, top=10, normalize=True, percent=False):
    vc = s.value_counts(ascending=False, normalize=normalize)
    vc.index = [truncate_str(i) for i in vc.index]
    out = vc[:top]
    if vc.sum() > out.sum():
        other = pd.Series([vc.sum() - out.sum()], index=['[OTROS] (%d tipos)' % (len(vc) - len(out))])
        out = out.append(other)
    na = 1.0 - vc.sum()
    out = out.append(pd.Series([na], index=['[NA]']))
    if percent:
        return out.apply(lambda p: '%.2f%%' % (p*100))
    return out

smart_value_counts(df[df.columns[0]], top=5, normalize=True, percent=True)

2010-2011            17.49%
2011-2012            14.89%
2013-2014            13.00%
2008-2009            12.77%
2007-2008            12.53%
[OTROS] (3 tipos)    29.31%
[NA]                  0.00%
dtype: object

In [7]:
# df.describe()

In [8]:
smart_value_counts(df['Campana'])

2010-2011    0.174941
2011-2012    0.148936
2013-2014    0.130024
2008-2009    0.127660
2007-2008    0.125296
2005-2006    0.101655
2009-2010    0.101655
2006-2007    0.089835
[NA]         0.000000
dtype: float64

In [30]:
saltados, numericos = 0, 0
for c in df.columns:
    if df[c].count() == 0:
        print '**SALTANDO "%s"** (todos NA) |' % c,
        saltados += 1
        continue
    fig, ax = plt.subplots(1)
    ax.set_title(c)
    print c + '',
    s = df[c]
    try:
        s = s.astype(float)
        s.hist(ax=ax, weights=np.zeros(s.count()) + 1. / len(s))
        ax.set_yticklabels(['%.0f%%' % (t*100) for t in ax.get_yticks()])
        na = 1 - float(s.count()) / len(s)
        if na > 0:
            ax.set_title(c + ' | NA = %.1f%%' % (na*100))
        print '(float) |',
        numericos += 1
    except ValueError:
        s = smart_value_counts(s)
        s[::-1].plot(ax=ax, kind='barh', color=[sns.color_palette()[2]] +
                     [sns.color_palette()[1]]*(len(s) - 1))
        ax.set_xticklabels(['%.0f%%' % (t*100) for t in ax.get_xticks()])
        print '(categorico) |',
    fig.subplots_adjust(left=0.2)
    figname = ''.join([ch for ch in c.lower().replace(' ', '_')
                       if ch.isalnum() or ch in ('_-')])
    fig.savefig('figuras/explorar_hist_%s.png' % figname)
    plt.close()

print '\n'
print 'Columnos: [%d numericos, %d categoricos, %d saltados (vacios), %d total]' % (
    numericos, len(df.columns) - numericos - saltados, saltados, len(df.columns))

Campana (categorico) | Empresa (categorico) | Establecimiento (categorico) | Sector (categorico) | Lote (categorico) | Superficie sembrada (has) (float) | Superficie cosechada (has) (float) | Antecesor (categorico) | Variedad (categorico) | Sistema de Laboreo (categorico) | Fecha de Siembra (categorico) | Densidad de Siembra (kg/ha) (float) | Fecha de Emergencia (70%) (categorico) | Densidad  (Plantas/m2) (categorico) | modalidad (categorico) | Tipo/nombre (categorico) | kg/ha Producto comercial (float) | modalidad.1 (categorico) | Tipo/nombre.1 (categorico) | kg/ha Producto comercial.1 (float) | modalidad.2 (categorico) | Tipo/nombre.2 (categorico) | kg/ha Producto comercial.2 (float) | N (float) | P (float) | K (float) | Fecha aplicacion prerriego (categorico) | Tipo (categorico) | N.1 (float) | **SALTANDO "K.1"** (todos NA) | **SALTANDO "S u otro"** (todos NA) | Fecha aplicacion macollaje (categorico) | N.2 (float) | **SALTANDO "K.2"** (todos NA) | **SALTANDO "S u otro.1"** (todos N

In [31]:
sumario = df.describe()
sumario

,Superficie sembrada (has),Superficie cosechada (has),Densidad de Siembra (kg/ha),kg/ha Producto comercial,kg/ha Producto comercial.1,kg/ha Producto comercial.2,N,P,K,N.1,K.1,S u otro,N.2,K.2,S u otro.1,N.3,K.3,S u otro.2,K.4,Dias emer-gencia-DPF,Toxicidad de Hierro,Emerg.-Lote Completo Dias,Dosis.3,Dosis (g o cm3/ha),Dias emergencia a fin de desecamiento,Emerg.50% Florac. Dias,Panojas/m2,Granos/Panoja,Ciclo = Dias de Emerg.-Cosecha,Humedad de Cose-cha (%),Total kg. Secos,Rend. Seco (Kg/ha),Eficiencia de produccion,Factor (%),Total Kg. con Factor,Rinde con Factor (kg/ha)
count,423.000000,423.000000,423.000000,166.000000,25.000000,107.000000,372.000000,372.000000,361.000000,311.00000,0,0,12,0,0,250.000000,0,0,114.000000,249.00000,0,385.000000,11.000000,19.000000,75.000000,423.000000,0,5.000000,423.000000,382.000000,423.000000,423.000000,423.000000,327.000000,368.000000,368.000000
mean,38.657352,38.653097,103.966903,112.680723,70.400000,87.765421,10.828387,23.075827,37.780709,47.16701,NaN,NaN,23,NaN,NaN,32.572800,NaN,NaN,54.433368,52.48996,NaN,26.677922,3.454545,0.842105,-35731.186667,90.846336,NaN,115.020000,-3385.257683,6.112495,268715.375153,7006.091686,0.150338,0.970690,235665.909311,6241.977735
std,17.409913,17.409731,14.632750,35.974071,11.895377,37.307643,6.210559,13.491036,19.829912,10.68954,NaN,NaN,0,NaN,NaN,10.774992,NaN,NaN,26.251850,6.01216,NaN,7.775506,0.756787,0.238783,10623.802996,8.282485,NaN,16.658691,11205.996192,9.191634,126909.427994,1451.891081,0.039042,0.090794,147892.369545,2635.367443
min,0.900000,0.900000,45.000000,45.000000,50.000000,36.000000,3.700000,8.720000,2.490000,0.00000,NaN,NaN,23,NaN,NaN,23.000000,NaN,NaN,24.900000,40.00000,NaN,2.000000,2.500000,0.500000,-39060.000000,72.000000,NaN,95.600000,-39060.000000,0.120000,6772.137931,2478.160920,0.091698,0.523500,0.000000,0.000000
25%,27.700000,27.700000,98.000000,100.000000,75.000000,58.000000,7.000000,13.080000,29.880000,46.00000,NaN,NaN,23,NaN,NaN,27.600000,NaN,NaN,39.840000,45.00000,NaN,22.000000,2.500000,0.500000,-39002.000000,84.500000,NaN,102.600000,121.000000,0.193000,183385.747127,6010.236697,0.123959,0.933000,130404.263250,5226.336143
50%,37.000000,37.000000,100.000000,120.000000,75.000000,94.300000,7.500000,19.349680,35.400000,46.00000,NaN,NaN,23,NaN,NaN,27.600000,NaN,NaN,39.840000,53.00000,NaN,26.000000,4.000000,1.000000,-38992.000000,91.000000,NaN,113.400000,130.000000,0.225000,257390.114943,7158.106371,0.139702,0.997500,226369.170238,6967.288222
75%,47.400000,47.400000,110.000000,150.000000,80.000000,130.000000,20.000000,23.544000,43.160000,46.00000,NaN,NaN,23,NaN,NaN,36.800000,NaN,NaN,80.000000,58.00000,NaN,32.000000,4.000000,1.000000,-38635.000000,97.000000,NaN,128.900000,137.000000,17.000000,342098.172414,8067.185152,0.166383,1.030500,327449.066250,8118.232720
max,123.000000,123.000000,162.000000,250.000000,80.000000,140.000000,25.200000,63.000000,86.320000,101.20000,NaN,NaN,23,NaN,NaN,69.000000,NaN,NaN,100.000000,64.00000,NaN,54.000000,4.000000,1.000000,50.000000,112.000000,NaN,134.600000,163.000000,26.000000,796757.000000,10905.340622,0.403525,1.101000,808309.976500,10981.678007


In [11]:
# for i, col in enumerate(df.columns):
#     if col in summario.columns:
#         print 'Saltando %s' % col
#         continue
#     print '*' * 10 + ' %d: %s ' % (i, col) + '*' * 10
#     s = smart_value_counts(df[col], top=5, normalize=True, percent=True)
#     print s